## Packages

In [1]:
import numpy as np
np.set_printoptions(precision=4, suppress=True, linewidth=200)
np.random.seed(2018)
import pandas as pd
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 6.0) #set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
sns.set(color_codes=True)
np.set_printoptions(precision=10, suppress=True, linewidth=200, threshold=1000, edgeitems=25)
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1", "2' "3" "4"
os.environ["CUDA_VISIBLE_DEVICES"]= ''
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))

import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import Normalizer, MinMaxScaler

from scipy.cluster.vq import kmeans2 as kmeans

In [3]:
import pickle

## Check Data (pandas)

In [ ]:
df_airline = pd.read_csv('./data/airplane.csv', header=None)
cols = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']

In [ ]:
df_airline = df_airline.values
df_airline = pd.DataFrame(df_airline, columns=cols)

In [ ]:
df_airline.describe()

In [ ]:
_, clusters = kmeans(df_airline.values.astype('float32'), 1000, minit='points')

In [ ]:
df_airline['cluster'] = pd.Series(clusters, index=df_airline.index)

In [ ]:
X = df_airline.iloc[:, :-2]
y = df_airline.iloc[:, -2:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=2018)

In [ ]:
df_airline_train, df_airline_test = train_test_split(df_airline, test_size=0.01, random_state=2018)

In [ ]:
df_airline_train.shape, df_airline_test.shape

In [ ]:
normalizer = StandardScaler()

In [ ]:
df_train_norm = normalizer.fit_transform(df_airline_train.values[:, :-1]) # no cluster
df_test_norm = normalizer.fit_transform(df_airline_test.values[:, :-1])  # no cluster

In [ ]:
np.column_stack((df_train_norm, df_airline_train.cluster.values))

In [ ]:
#
normalizer_x = StandardScaler()
normalizer_y = StandardScaler()

X_train_norm = normalizer_x.fit_transform(X_train)
y_train_norm = normalizer_y.fit_transform(y_train.values[:, None])

X_test_norm = normalizer_x.fit_transform(X_test)
y_test_norm = normalizer_y.fit_transform(y_test.values[:, None])

## Load dataloader

In [7]:
import data_loader

In [8]:
dtrain, dtest, z, y_std = data_loader.load('airplane.csv', 
                                           n_clusters=10, 
                                           n_induce=10)

loading data of 2055732 datapoints...
Parition into 10 clusters...
Done...!
Selecting 10 inducing variables...
Done...!


## VBSGPR

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [ ]:
import tensorflow as tf
import numpy as np
import random
from model import VBSGPR

epochs = 10
N, total_dim = dtrain.shape
log_beta, log_sf2, log_theta = 0., 0., 0.
clusters = [i for i in range(100)]

In [ ]:
model = VBSGPR(N, log_beta_opt, log_sf2_opt, log_theta_opt, z_opt,  qmu_opt, qlogdev_opt)

In [ ]:
lb = model.lower_bound()
fmu, fcov = model.predict_f()
gp_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'vbsgpr')
gp_opt = tf.train.AdamOptimizer(0.01, name='gp_opt')
gp_train_op = gp_opt.minimize(-lb, var_list=gp_vars)

In [ ]:
sess.run(tf.global_variables_initializer())
f_test, _ = sess.run([fmu, fcov], {model.x: X_test})
rmse = np.sqrt(np.mean(y_std**2 * ((y_test - f_test))**2))
print ('test RMSE: {}'.format(rmse))

In [ ]:
model.qlogdev.eval()

In [ ]:
# with tf.Session() as sess:
sess.run(tf.global_variables_initializer())
for epoch in range(epochs):
    random.shuffle(clusters)
    for i, cluster in enumerate(clusters):
        data_batch = dtrain[np.where(dtrain[:, -1] == cluster)]
        X, y = data_batch[:, :-2], data_batch[:, -2:-1]
        _, lb_ = sess.run([gp_train_op, lb], {model.x: X, model.y: y, model.batch: y.shape[0]})
        if i % 50 == 0: 
            print ('Epoch: [{}], The {}-th Cluster: [{}], Lower Bound: [{}]'.format(
                    epoch, i, cluster, lb_))
            X_test, y_test = dtest[:, :-2], dtest[:, -2:-1]
            f_test, _ = sess.run([fmu, fcov], {model.x: X_test})
            rmse = np.sqrt(np.mean(y_std**2 * ((y_test - f_test))**2))
            print ('Epoch {} test RMSE: {}'.format(epoch, rmse))

## SVGP

In [24]:
import gpflow
from gpflow.models import SVGP
from gpflow.likelihoods import Gaussian
from gpflow.kernels import RBF
from gpflow.training import AdamOptimizer

In [25]:
likelihood = Gaussian(variance=0.5)
kern = RBF(8, variance=2., lengthscales=2., ARD=False)

In [26]:
X_train, y_train = dtrain[:, :-2], dtrain[:, -2:-1]
X_test, y_test = dtest[:, :-2], dtest[:, -2:-1]

In [27]:
np.random.seed(2018)
# qmu = np.random.randn(1, 1)
qmu = np.random.randn(10, 1)

In [28]:
svgp = SVGP(X_test, y_test, kern, 
            likelihood, z, minibatch_size=y_test.shape[0], 
            q_diag=True, q_mu=qmu, whiten=True)

In [29]:
# optimize
AdamOptimizer(0.01).minimize(svgp, maxiter=1000)

In [30]:
svgp.compute_log_likelihood()

-28920.051483041836

In [31]:
yhat_test = svgp.predict_y(X_test)[0]
np.sqrt(np.mean(y_std**2 * ((y_test - yhat_test))**2))

40.05194496807794

In [32]:
z_opt = svgp.feature.Z.value
log_sf2_opt = np.log(svgp.kern.variance.value)
log_theta_opt = np.log(svgp.kern.lengthscales.value)
log_beta_opt = np.log(1. / svgp.likelihood.variance.value)
qmu_opt = svgp.q_mu.value
qlogdev_opt = np.log(svgp.q_sqrt.value[:, 0])

In [33]:
dic = dict(z_opt=z_opt, log_beta_opt=log_beta_opt, 
           log_sf2_opt=log_sf2_opt, log_theta_opt=log_theta_opt,
           qmu_opt=qmu_opt, qlogdev_opt=qlogdev_opt)

In [34]:
with open('svgp.pkl', 'wb') as f:
    pickle.dump(dic, f)

In [35]:
svgp.as_pandas_table()

,class,prior,transform,trainable,shape,fixed_shape,value
SVGP/likelihood/variance,Parameter,None,+ve,True,(),True,0.9916825474228069
SVGP/kern/variance,Parameter,None,+ve,True,(),True,0.5779710842721315
SVGP/kern/lengthscales,Parameter,None,+ve,True,(),True,4.771515422576705
SVGP/q_mu,Parameter,None,(none),True,"(10, 1)",True,"[[-0.0659091363619212], [0.07995137005926004],..."
SVGP/feature/Z,Parameter,None,(none),True,"(10, 8)",True,"[[-0.015726817537224005, 0.06890777138039703, ..."
SVGP/q_sqrt,Parameter,None,+ve,True,"(10, 1)",True,"[[0.05633904389218809], [0.2304703386384144], ..."


In [ ]:
AdamOptimizer()